In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"
os.environ["http_proxy"] = "http://10.134.231.119:7890"
os.environ["https_proxy"] = "http://10.134.231.119:7890"

# 检查 GPU 是否可用  
import torch
if torch.cuda.is_available():  
   print("GPU 可用")  
   # 获取 GPU 设备数量  
   num_gpus = torch.cuda.device_count()  
   print(f"共有{num_gpus}个 GPU 设备")
   # 获取 GPU 设备信息  
   for i in range(num_gpus):  
       print(f"设备{i}: {torch.cuda.get_device_properties(i)}")  
else:  
   print("GPU 不可用")  
import sys
# sys.path.append("/workspace/notebooks/")
# sys.path.append("/workspace/notebooks/FSS/")
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

GPU 可用
共有1个 GPU 设备
设备0: _CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=11011MB, multi_processor_count=68)


# Bert model
pip install transformers

Experimental results of bert-base-cased on GLUE 
| Task  | Metric                       | Result      | Training time | Result (FP16) | Training time (FP16) |
|-------|------------------------------|-------------|---------------|---------------|----------------------|
| CoLA  | Matthews corr                | 56.53       | 3:17          | 56.78         | 1:41                 |
| SST-2 | Accuracy                     | 92.32       | 26:06         | 91.74         | 13:11                |
| MRPC  | F1/Accuracy                  | 88.85/84.07 | 2:21          | 88.12/83.58   | 1:10                 |
| STS-B | Pearson/Spearman corr.       | 88.64/88.48 | 2:13          | 88.71/88.55   | 1:08                 |
| QQP   | Accuracy/F1                  | 90.71/87.49 | 2:22:26       | 90.67/87.43   | 1:11:54              |
| MNLI  | Matched acc./Mismatched acc. | 83.91/84.10 | 2:35:23       | 84.04/84.06   | 1:17:06              |
| QNLI  | Accuracy                     | 90.66       | 40:57         | 90.96         | 20:16                |
| RTE   | Accuracy                     | 65.70       | 57            | 65.34         | 29                   |
| WNLI  | Accuracy                     | 56.34       | 24            | 56.34         | 12                   |

# Training Bert with PyTorch

## 1、Get datasets and tokenizer, processing datasets

In [2]:
# pip install datasets transformers
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_datasets = tokenized_datasets.remove_columns(['sentence1', 'sentence2','idx'])
# tokenized_datasets = tokenized_datasets.rename_column('label','labels')  # 实践证明，这一行是不需要的
tokenized_datasets.set_format('torch')

from torch.utils.data import DataLoader, Dataset
# 通过这里的dataloader，每个batch的seq_len可能不同
train_dataloader = DataLoader(tokenized_datasets['train'],
                              shuffle=True, batch_size=64,
                              collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets['validation'],
                             batch_size=64,
                             collate_fn=data_collator)

In [3]:
# MTL-Bert Model
import torch
from transformers import AutoModelForSequenceClassification,BertForSequenceClassification
import sys
sys.path.append('/home/pytorchtest/notebooks/fss')
from src.dns_bert import MTLBertForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
if isinstance(pretrained_model,BertForSequenceClassification):
    # model = MTLBertForSequenceClassification(pretrained_model.config).to(device)
    model = MTLBertForSequenceClassification(model.config,use_dns=True,dns_ratio=0.7).to(device)
    model.bert.load_state_dict(pretrained_model.bert.state_dict())
else:
    raise ValueError("pretrained_model must be BertForSequenceClassification")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import torch
from transformers import AdamW, get_scheduler

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # num of batches * num of epochs
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,  # scheduler是针对optimizer的lr的
    num_warmup_steps=0,
    num_training_steps=num_training_steps)
print(num_training_steps)

174


In [5]:
from tqdm import tqdm
import evaluate
import time
import torch

class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
def accuracy_predictions(predictions,references):
    return torch.sum(predictions==references)/len(references)
def accuracy_logits(logits,references):
    predictions = torch.argmax(logits, dim=-1)
    return accuracy_predictions(predictions,references)

metric_res = []*model.config.num_hidden_layers
print_freq = 20

for epoch in range(num_epochs):

  batch_time = AverageMeter()
  data_time = AverageMeter()
  losses = AverageMeter()
  acces = []
  for i in range(model.config.num_hidden_layers):
      acces.append(AverageMeter())

  # training
  model.train()
  # tbar = tqdm(train_dataloader,unit="batch",desc="Epoch %d Training:"%epoch)
  print(f"#############################Epoch {epoch+1}#######################")
  print(f"lr = {optimizer.param_groups[0]['lr']}")
  end = time.time()
  for idx, batch in enumerate(train_dataloader):
      data_time.update(time.time() - end)
      # 要在GPU上训练，需要把数据集都移动到GPU上：
      batch = {k:v.to(device) for k,v in batch.items()}
      outputs = model(**batch)
      loss=sum(outputs.loss)
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      optimizer.zero_grad()
      # measure elapsed time
      batch_time.update(time.time() - end)
      end = time.time()

      losses.update(loss.item(), batch["labels"].size(0))
      for i in range(model.config.num_hidden_layers):
        acc = accuracy_logits(outputs.logits[i],batch["labels"])
        acces[i].update(acc.item(), batch["labels"].size(0))

      if idx%print_freq==0:
        # 先使用最后一层输出的结果训练看能达到之前的结果吗
        msg_info = f"loss:{losses.avg:.4f}, | Time: {batch_time.avg:.4f}, Data: {data_time.avg:.4f} | "
        for i in range(model.config.num_hidden_layers):
          msg_info += f"Acc {i+1}/{model.config.num_hidden_layers}: {acces[i].avg:.4f},"
        print(f"Training Epoch [{epoch}/{num_epochs}] Iter [{idx}/{len(train_dataloader)}]: ",msg_info)

  # evaluation
  val_losses = AverageMeter()
  val_acces = []
  for i in range(model.config.num_hidden_layers):
      val_acces.append(AverageMeter())
  model.eval()
  for idx, batch in enumerate(eval_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():  # evaluation的时候不需要算梯度
      outputs = model(**batch)
    loss=sum(outputs.loss)
    val_losses.update(loss.item(), batch["labels"].size(0))
    for i in range(model.config.num_hidden_layers):
      logits = outputs.logits[i]
      predictions = torch.argmax(logits, dim=-1)
      acc = accuracy_predictions(predictions,batch["labels"])
      val_acces[i].update(acc.item(), batch["labels"].size(0))
      # metric_objs[i].add_batch(predictions=predictions, references=batch["labels"])

    if idx%print_freq==0:
      msg_info = f"loss:{val_losses.avg:.4f}, "
      for i in range(model.config.num_hidden_layers):
          msg_info += f"Acc {i+1}/{model.config.num_hidden_layers}: {val_acces[i].avg:.4f},"
      print(f"Evaluation Epoch [{epoch}/{num_epochs}] Iter [{idx}/{len(eval_dataloader)}]:",msg_info)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


#############################Epoch 1#######################
lr = 5e-05
Training Epoch [0/3] Iter [0/58]:  loss:8.1469, | Time: 1.9415, Data: 0.1041 | Acc 1/12: 0.6719,Acc 2/12: 0.4219,Acc 3/12: 0.6562,Acc 4/12: 0.6250,Acc 5/12: 0.6406,Acc 6/12: 0.6562,Acc 7/12: 0.3750,Acc 8/12: 0.6562,Acc 9/12: 0.3125,Acc 10/12: 0.6562,Acc 11/12: 0.4375,Acc 12/12: 0.6094,
Training Epoch [0/3] Iter [20/58]:  loss:7.4038, | Time: 0.3921, Data: 0.0399 | Acc 1/12: 0.6763,Acc 2/12: 0.6629,Acc 3/12: 0.6756,Acc 4/12: 0.6741,Acc 5/12: 0.6741,Acc 6/12: 0.6868,Acc 7/12: 0.6749,Acc 8/12: 0.6845,Acc 9/12: 0.6763,Acc 10/12: 0.6868,Acc 11/12: 0.6696,Acc 12/12: 0.6823,
Training Epoch [0/3] Iter [40/58]:  loss:7.1952, | Time: 0.3566, Data: 0.0382 | Acc 1/12: 0.6787,Acc 2/12: 0.6719,Acc 3/12: 0.6784,Acc 4/12: 0.6806,Acc 5/12: 0.6871,Acc 6/12: 0.7031,Acc 7/12: 0.6955,Acc 8/12: 0.7012,Acc 9/12: 0.6936,Acc 10/12: 0.7005,Acc 11/12: 0.6894,Acc 12/12: 0.6982,
Evaluation Epoch [0/3] Iter [0/7]: loss:5.6967, Acc 1/12: 0.6719,A

In [6]:
# evaluation
metric_objs = [evaluate.load("glue", "mrpc") for _ in range(model.config.num_hidden_layers)]
val_losses = AverageMeter()
val_acces = []
print_freq = 1
for i in range(model.config.num_hidden_layers):
    val_acces.append(AverageMeter())
model.eval()
for idx,batch in enumerate(eval_dataloader):
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():  # evaluation的时候不需要算梯度
    outputs = model(**batch)
  loss=sum(outputs.loss)
  val_losses.update(loss.item(), batch["labels"].size(0))
  for i in range(model.config.num_hidden_layers):
    logits = outputs.logits[i]
    predictions = torch.argmax(logits, dim=-1)
    acc = accuracy_predictions(predictions,batch["labels"])
    val_acces[i].update(acc.item(), batch["labels"].size(0))
    metric_objs[i].add_batch(predictions=predictions, references=batch["labels"])

  if idx%print_freq==0:
    msg_info = f"loss:{val_losses.avg:.4f}, "
    for i in range(model.config.num_hidden_layers):
        msg_info += f"Acc {i+1}/{model.config.num_hidden_layers}: {val_acces[i].avg:.4f},"
    print(f"Evaluation Epoch [{epoch}/{num_epochs}] Iter [{idx}/{len(eval_dataloader)}]:",msg_info)

for i in range(len(metric_objs)):
  metric_res.append(metric_objs[i].compute())

bert_base_base_res_on_mrpc = [f"{i['f1']*100:.2f}/{i['accuracy']*100:.2f}" for i in metric_res]
for i in bert_base_base_res_on_mrpc:
  print(i,"|",end="  ")

Evaluation Epoch [2/3] Iter [0/7]: loss:4.5917, Acc 1/12: 0.6719,Acc 2/12: 0.7031,Acc 3/12: 0.7500,Acc 4/12: 0.7812,Acc 5/12: 0.8281,Acc 6/12: 0.8906,Acc 7/12: 0.8750,Acc 8/12: 0.8750,Acc 9/12: 0.8750,Acc 10/12: 0.9062,Acc 11/12: 0.9219,Acc 12/12: 0.9219,
Evaluation Epoch [2/3] Iter [1/7]: loss:5.0606, Acc 1/12: 0.7031,Acc 2/12: 0.7266,Acc 3/12: 0.7500,Acc 4/12: 0.8047,Acc 5/12: 0.8281,Acc 6/12: 0.8516,Acc 7/12: 0.8438,Acc 8/12: 0.8516,Acc 9/12: 0.8438,Acc 10/12: 0.8750,Acc 11/12: 0.8828,Acc 12/12: 0.8906,
Evaluation Epoch [2/3] Iter [2/7]: loss:5.6142, Acc 1/12: 0.6823,Acc 2/12: 0.7031,Acc 3/12: 0.7292,Acc 4/12: 0.7760,Acc 5/12: 0.7917,Acc 6/12: 0.8125,Acc 7/12: 0.8177,Acc 8/12: 0.8333,Acc 9/12: 0.8281,Acc 10/12: 0.8438,Acc 11/12: 0.8542,Acc 12/12: 0.8594,
Evaluation Epoch [2/3] Iter [3/7]: loss:5.7743, Acc 1/12: 0.6953,Acc 2/12: 0.6953,Acc 3/12: 0.7227,Acc 4/12: 0.7773,Acc 5/12: 0.7852,Acc 6/12: 0.7969,Acc 7/12: 0.8008,Acc 8/12: 0.8125,Acc 9/12: 0.8125,Acc 10/12: 0.8203,Acc 11/12: 0.